### Imports

In [ ]:
# Import necessary libraries
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import io, transform
from skimage.util import random_noise
from skimage.transform import rotate, AffineTransform, warp
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Config

In [ ]:
# Set seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# 1. Data Preparation

In [ ]:
# Assuming 'data/train' is the directory where your training images are stored
data_train = 'data/train'

def augment_images(image_paths, n_augmented=20, output_size=(64, 64)):
    augmented_images = []
    
    # Randomly select 'n_augmented' images to augment
    selected_image_paths = random.sample(image_paths, n_augmented)
    
    for image_path in selected_image_paths:
        # Load the image
        image = io.imread(image_path)

        # Random 3D rotation (simulated with 2D rotation for simplicity)
        angle = random.uniform(-5, 5)  # Random angle between -15 and 15 degrees
        rotated_image = transform.rotate(image, angle, mode='wrap')

        # Add random noise
        noisy_image = random_noise(rotated_image, mode='gaussian')

        # Resize the image
        resized_image = transform.resize(noisy_image, output_size, anti_aliasing=True)

        # Convert the image data type to uint8
        final_image = (noisy_image * 255).astype(np.uint8)
        
        augmented_images.append(final_image)

    return augmented_images

# Get a list of image paths from the training directory
image_paths = [os.path.join(data_train, filename) for filename in os.listdir(data_train) if filename.endswith('.png')]

# Apply the 'augment_images' function to augment 20 images
augmented_validation_images = augment_images(image_paths, n_augmented=20)

# Save the augmented images to the 'data/validate' directory
os.makedirs('data/validate', exist_ok=True)  # Make sure the validation directory exists
for i, image in enumerate(augmented_validation_images):
    io.imsave(f'data/validate/aug_{i}.png', image)

In [ ]:
def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder, filename))
        if img is not None:
            # Keep only the first three channels (RGB)
            img = img[:, :, :3]
            images.append(img)
            filenames.append(filename)
    return images, filenames

def preprocess_images(images, size=(64, 64)):
    processed_images = []
    for img in images:
        # Ensure the image has three channels
        if img.shape[-1] != 3:
            continue
        img_resized = transform.resize(img, size, anti_aliasing=True)
        img_normalized = img_resized / 255.0
        processed_images.append(img_normalized)
    return np.array(processed_images)

# Load and preprocess images
images, filenames = load_images_from_folder('data/train')
images = preprocess_images(images)


# Task 2: CNN Architecture and Transfer Learning


### 2.1. Develop three different architectures from scratch

In [ ]:
# 2. CNN Architecture and Transfer Learning
def create_model_architecture(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

# Create different architectures
model_architectures = [create_model_architecture(images.shape[1:]) for _ in range(3)]

### 2.2. Transfer learning for each model

In [ ]:
# 2. CNN Architecture and Transfer Learning
def create_model_architecture(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return model

# Create different architectures
model_architectures = [create_model_architecture(images.shape[1:]) for _ in range(3)]
create_model_architecture(images.shape[1:]).save('models/transfer_model_{i}.keras', i)

In [ ]:
# Transfer learning for each model
def apply_transfer_learning(model):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    base_model.trainable = False

    # Create a new model with the base model and custom layers
    new_model = Sequential([
        base_model,
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    return new_model

# Apply transfer learning and inspect the model summary
transfer_model = apply_transfer_learning(create_model_architecture((64, 64, 3)))
transfer_model.summary()

# Task 3: CNN Training

In [ ]:
# 3. CNN Training
def train_model(model, images, train_labels, epochs=10, batch_size=32):
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(images, train_labels, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    return history

# Prepare train labels for binary classification (assuming you have two classes)
train_labels = np.zeros(len(images))
# For example, if the first half of your images are class 0 and the second half are class 1
train_labels[len(images)//2:] = 1

# Train the transfer learning model
print("Training the transfer learning model...")
history = train_model(transfer_model, images, train_labels, epochs=10)

# Save the model
os.makedirs('models', exist_ok=True)
transfer_model.save('models/transfer_model.keras')

# Task 4: Hyperparameters and Model Selection

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Transfer Learning Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Transfer Learning Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()